# TAREA 3
# NLP2
# https://github.com/TextMiningNLP2/TAREA3/

In [61]:
# SE IMPORTAN LAS LIBRERIAS INSTALAS
# PREVIAMENTE SE HAN INSTALADO CIERTAS LIBERÍAS EN EL VENV

import enum
import pickle
from pprint import pprint
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import random
from typing import Generator, Dict, List, Tuple
from gensim.corpora import Dictionary
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import spacy
from spacy.lang.en import English
from spacy.lang.es import Spanish
import nltk


In [62]:
# De ser necesario, descargar en Corpora / Stop_words
# Si se desea se puede descargar all_corpora pero toma tiempo
# nltk.download()

In [83]:
# CLASE DEFINIDORA DE SENTIMIENTOS
# obtenida del ejemplo prepare_data

class Sentiments(enum.Enum):
    POS = 'POS'
    NEG = 'NEG'

# FUNCION PARA DIVIDIR DATA
# obtenida del ejemplo train

def split_data(data: List, weights: List = (0.7, 0.15, 0.15)):
    split = {
        'train': [],
        'test': [],
        'validation': [],
    }
    for word in data:
        subset = random.choices(['train', 'test', 'validation'], weights=weights)[0]
        split[subset].append(word)

    return split

# FUNCION CONVERTIR ORACIONES A PALABRAS
# obtenida del ejemplo tokenize

def sentences_to_words(sentences: List[str]) -> List[List[str]]:
    words = []
    for sentence in sentences:
        # https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess
        words.append(simple_preprocess(str(sentence), deacc=True))  # deacc=True elimina la puntuación
    return words

# FUNCION QUE REMUEVE LOS STOPWORDS DADOS
# obtenida del ejemplo tokenize

def remove_stopwords(documents: List[List[str]]) -> List[List[str]]:
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords.words('english')]
            for doc in documents]

# FUNCION PARA APRENDER BIGRAMAS
# obtenida del ejemplo tokenize

def learn_bigrams(documents: List[List[str]]) -> List[List[str]]:
    # We learn bigrams
    #  https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.Phrases
    bigram = Phrases(documents, min_count=5, threshold=10)

    # we reduce the bigram model to its minimal functionality
    bigram_mod = Phraser(bigram)

    # we apply the bigram model to our documents
    return bigram_mod

# FUNCION PARA CREAR BIGRAMAS
# obtenida del ejemplo tokenize

def create_bigrams(bigram_model, documents: List[List[str]]):
    return [bigram_model[doc] for doc in documents]

# FUNCION PARA LEMATIZAR
# obtenida del ejemplo tokenize

def lemmatization(nlp: English, texts: List[List[str]], allowed_postags: List = None) -> List[List[str]]:
    if allowed_postags is None:
        allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV']

    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# FUNCION PARA TOKENIZAR
# Resume cada uno de los pasos del pipeline de pre procesamiento
# obtenida del ejemplo tokenize
 
def tokenize(documents: List[str], bigram_model) -> List[List[str]]:

    document_words = sentences_to_words(documents)
    document_words = remove_stopwords(document_words)
    document_words = create_bigrams(bigram_model, document_words)
    document_words = lemmatization(nlp, document_words)

    return document_words

# FUNCION QUE CREA DICCIONARIO
# Obtenida del ejemplo train

def create_dictionary(documents: List[List[str]]):
    return Dictionary(documents)


In [64]:
df = pd.read_csv('../data/raw/reviews.csv')

df['rating'] = df['rating'].astype(dtype='int64')
df['sentiment'] = df['rating'].apply(lambda x: Sentiments.POS if x >= 40 else Sentiments.NEG)

data_classes = {sentiment.value: df[df['sentiment'] == sentiment]['review'].values.tolist() for sentiment in Sentiments}

pos_rev = data_classes['POS']
neg_rev = data_classes['NEG']

splitPOS = split_data(pos_rev)
splitNEG = split_data(neg_rev)


In [65]:
# split_data(pos_rev)['train'][0]

In [66]:
# pprint(data_classes['NEG'])

In [67]:
# PARA GUARDAR ARCHIVO BINARIO DE PYTHON
# NO ES NECESARIO CORRER, YA ESTA GUARDADA UNA COPIA

with open(r"../data/interim/review_classes.pkl", "wb") as output_file:
    pickle.dump(data_classes, output_file)

In [68]:
# PARA LEER ARCHIVO BINARIO DE PYTHON
# CORRER ESTE PARA LEER LA COPIA GUARDADA

with open(r"../data/interim/review_classes.pkl", "rb") as input_file:
    review_classes = pickle.load(input_file)

In [69]:
# SE DIVIDEN LAS CLASES EN POSITIVAS Y NEGATIVAS

positive_reviews = review_classes['POS']
negative_reviews = review_classes['NEG']

In [70]:
# SE DIVIDEN LAS CLASES EN PALABRAS UNICAS
# NO ES NECESARIO CORRER
# ABAJO LO TENEMOS PROCESADO, SOLO HACER ELCTURA

# print(negative_reviews[1])
negative_words = sentences_to_words(negative_reviews)
positive_words = sentences_to_words(positive_reviews)
# print(negative_words[1])

In [71]:
# PARA GUARDADO DE NEGATIVE/POSITIVE WORDS
# NO ES NECESARIO CORRER, YA QUE SE ENTREGAN LOS ARCHIVOS YA GUARDADOS
# SALTAR A LA SIGUIENTE CELDA
with open(r"../data/interim/negative_words.pkl", "wb") as output_file:
    pickle.dump(data_classes, output_file)
    
with open(r"../data/interim/positive_words.pkl", "wb") as output_file:
    pickle.dump(data_classes, output_file)

In [72]:
# PARA LECTURA DE LOS NEGATIVE/POSITIVE WORDS YA PROCESADO
# No se gasta tiempo en procesarlo otra vez

with open(r"../data/interim/review_classes.pkl", "rb") as input_file:
    negative_words = pickle.load(input_file)

with open(r"../data/interim/review_classes.pkl", "rb") as input_file:
    positive_words = pickle.load(input_file)

In [73]:
# DE LAS VARIABLES ANTERIORES, SE ELIMINAN LOS STOPWORDS

# print(negative_words[1])
negative_words_wo_stopwords = remove_stopwords(negative_words)
# print(negative_words_wo_stopwords[1])
positive_words_wo_stopwords = remove_stopwords(positive_words)

In [74]:
# SE CREA MODELO DE BIGRAMAS
# A partir de TODO el corpus procesado

bigram_model = learn_bigrams(negative_words_wo_stopwords + positive_words_wo_stopwords)
negative_words_wo_stopwords_bigrams = create_bigrams(bigram_model, negative_words_wo_stopwords)

In [75]:
# nlp = spacy.load("en_core_web_trf")
nlp = spacy.load("es_dep_news_trf",disable=['parser','ner'])
# nlp = spacy.load('es_dep_news_trf')
# nlp = spacy.load("en_core_web_sm")

ValueError: [E002] Can't find factory for 'transformer' for language Spanish (es). This usually happens when spaCy calls `nlp.create_pipe` with a custom component name that's not registered on the current language class. If you're using a Transformer, make sure to install 'spacy-transformers'. If you're using a custom component, make sure you've added the decorator `@Language.component` (for function components) or `@Language.factory` (for class components).

Available factories: attribute_ruler, tok2vec, merge_noun_chunks, merge_entities, merge_subtokens, token_splitter, doc_cleaner, parser, beam_parser, lemmatizer, trainable_lemmatizer, entity_linker, ner, beam_ner, entity_ruler, tagger, morphologizer, senter, sentencizer, textcat, spancat, future_entity_ruler, span_ruler, textcat_multilabel, en.lemmatizer, es.lemmatizer

In [76]:
negative_words_wo_stopwords_bigrams_pos = lemmatization(English, texts=negative_words_wo_stopwords_bigrams)
# negative_words_wo_stopwords_bigrams_pos = lemmatization(nlp, negative_words_wo_stopwords_bigrams)

ValueError: [E918] Received invalid value for vocab: pos (<class 'type'>). Valid values are an instance of `spacy.vocab.Vocab` or True to create one (default).

In [78]:
positive_words = tokenize(positive_reviews, bigram_model)

NameError: name 'nlp' is not defined

In [ ]:
negative_words = tokenize(negative_reviews, bigram_model)

In [ ]:
with open(r"../data/interim/positive_words.pkl", "wb") as output_file:
    pickle.dump(positive_words, output_file)

with open(r"../data/interim/negative_words.pkl", "wb") as output_file:
    pickle.dump(negative_words, output_file)
    

In [ ]:
with open(r"../data/interim/bigram_model.pkl", "wb") as output_file:
    pickle.dump(bigram_model, output_file)

In [79]:
with open(r"../data/interim/positive_words.pkl", "rb") as input_file:
    positive_docs = pickle.load(input_file)

In [80]:
with open(r"../data/interim/negative_words.pkl", "rb") as input_file:
    negative_docs = pickle.load(input_file)

In [81]:
negative_words = [item for sublist in negative_docs for item in sublist]
positive_words = [item for sublist in positive_docs for item in sublist]

In [84]:
dictionary = create_dictionary([negative_words, positive_words])

In [85]:
negative_docs_split = split_data(negative_docs)
positive_docs_split = split_data(positive_docs)

In [86]:
negative_words_split = {key: [item for sublist in value for item in sublist] for key, value in negative_docs_split.items()}
positive_words_split = {key: [item for sublist in value for item in sublist] for key, value in positive_docs_split.items()}

In [87]:
negative_bow = dictionary.doc2bow(negative_words_split['train'])
positive_bow = dictionary.doc2bow(positive_words_split['train'])

In [88]:
total_negative_words = len(negative_words_split['train']) + len(dictionary)
total_positive_words = len(positive_words_split['train']) + len(dictionary)

In [89]:
negative_word_probs = {}
for id, count in negative_bow:
    negative_word_probs[dictionary[id]] = {
        'id': id,
        'logprob': np.log((count + 1)/total_negative_words),
    }

negative_word_probs[-1] = {
    'id': -1,
    'logprob': np.log(1/total_negative_words)
}

In [90]:
positive_word_probs = {}
for id, count in positive_bow:
    positive_word_probs[dictionary[id]] = {
        'id': id,
        'logprob': np.log((count + 1)/total_positive_words),
    }
positive_word_probs[-1] = {
    'id': -1,
    'logprob': np.log(1/total_positive_words)
}

In [91]:
negative_prob = len(negative_docs_split["train"]) / (len(negative_docs_split["train"]) + len(positive_docs_split["train"]))
positive_prob = len(positive_docs_split["train"]) / (len(negative_docs_split["train"]) + len(positive_docs_split["train"]))

In [92]:
model = {
    'POS_PROB': np.log(positive_prob),
    'NEG_PROB': np.log(negative_prob),
    'COND_POS_PROBS': positive_word_probs,
    'COND_NEG_PROBS': negative_word_probs
}

C:\Users\chris\AppData\Local\Temp\ipykernel_21740\3345210171.py:2: RuntimeWarning: divide by zero encountered in log
  'POS_PROB': np.log(positive_prob),


In [95]:
with open(r"../models/model.pkl", "wb") as output_file:
    pickle.dump(model, output_file)

FileNotFoundError: [Errno 2] No such file or directory: '../models/model.pkl'

In [ ]:
model